<a href="https://colab.research.google.com/github/rex17/Machine-Learning-Practice/blob/master/Deep_Learning_Housing_Price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files
import numpy as np
import pandas as pd 
import math


from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [2]:
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving housing_df_cleaned.csv to housing_df_cleaned.csv
Saving test.csv to test.csv
User uploaded file "housing_df_cleaned.csv" with length 363604 bytes
User uploaded file "test.csv" with length 451405 bytes


In [0]:
housing_df = pd.read_csv("housing_df_cleaned.csv")
test_df = pd.read_csv('test.csv')


In [4]:
housing_df.shape

(1459, 63)

In [6]:
housing_df.head()

,Id,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,3,60,RL,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,4,70,RL,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,5,60,RL,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,12,2008,WD,Normal,250000


In [0]:
X = housing_df[["OverallQual","OverallCond","TotalBsmtSF","GrLivArea","1stFlrSF","GarageArea","LotArea","WoodDeckSF","EnclosedPorch","3SsnPorch","ScreenPorch","PoolArea","MiscVal","MoSold"]]

In [0]:
X_test = test_df[["OverallQual","OverallCond","TotalBsmtSF","GrLivArea","1stFlrSF","GarageArea","LotArea","WoodDeckSF","EnclosedPorch","3SsnPorch","ScreenPorch","PoolArea","MiscVal","MoSold"]]

In [54]:
X.head()

,OverallQual,TotalBsmtSF,GrLivArea,1stFlrSF,GarageArea,LotArea,WoodDeckSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold
0,7,856,1710,856,548,8450,0,0,0,0,0,0,2
1,6,1262,1262,1262,460,9600,298,0,0,0,0,0,5
2,7,920,1786,920,608,11250,0,0,0,0,0,0,9
3,7,756,1717,961,642,9550,0,272,0,0,0,0,2
4,8,1145,2198,1145,836,14260,192,0,0,0,0,0,12


In [0]:
Y = housing_df.iloc[:,62]

In [24]:
Y.head()

0    208500
1    181500
2    223500
3    140000
4    250000
Name: SalePrice, dtype: int64

In [0]:
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(14, input_dim=14, kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model

In [0]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset
estimator = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=5, verbose=0)

In [68]:
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(estimator, X, Y, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Results: -2246774045.23 (1039103375.45) MSE


In [58]:
estimator.fit(X, Y) # for 13 features

In [0]:
predictions = estimator.predict(X_test)

In [60]:
predictions

array([160520.02, 168914.69, 183748.3 , ..., 201681.52, 112469.26,
       216897.86], dtype=float32)

In [61]:
submission_df = pd.DataFrame(test_df['Id'],columns=['Id','SalePrice'])
submission_df["SalePrice"] = predictions
submission_df = submission_df[["Id","SalePrice"]]
submission_df.shape

(1459, 2)

In [62]:
submission_df.head()

,Id,SalePrice
0,1461,160520.015625
1,1462,168914.687500
2,1463,183748.296875
3,1464,189208.671875
4,1465,184272.546875


In [0]:
submission_df.to_csv("ss_submission_DL13.csv", index = False)